In [ ]:
# #import library yang dibutuhkan
# import pandas as pd
# import numpy as np



# #lakukan pembacaan dataset
# movie_rating_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/movie_rating_df.csv') #untuk menyimpan movie_rating_df.csv

# #tampilkan 5 baris teratas dari movive_rating_df
# print(movie_rating_df.head())

# #tampilkan info mengenai tipe data dari tiap kolom
# print(movie_rating_df.info())

# #Simpan actor_name.csv pada variable name_df 
# name_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/actor_name.csv')

# #Tampilkan 5 baris teratas dari name_df
# print(name_df.head())

# #Tampilkan informasi mengenai tipe data dari tiap kolom pada name_df
# print(name_df.info())

# #Menyimpan dataset pada variabel director_writers
# director_writers = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/directors_writers.csv')

# #Manampilkan 5 baris teratas
# print(director_writers.head())

# #Menampilkan informasi tipe data
# print(director_writers.info())

# import pandas as pd
# director_writers = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/directors_writers.csv')

# #Mengubah director_name menjadi list
# director_writers['director_name'] = director_writers['director_name'].apply(lambda row: row.split(','))
# director_writers['writer_name'] = director_writers['writer_name'].apply(lambda row: row.split(','))

# #Tampilkan 5 data teratas
# print(director_writers.head())

# name_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/actor_name.csv')
# #Kita hanya akan membutuhkan kolom nconst, primaryName, dan knownForTitles
# name_df = name_df[['nconst','primaryName','knownForTitles']]

# #Tampilkan 5 baris teratas dari name_df
# print(name_df.head())

# #Melakukan pengecekan variasi
# print(name_df['knownForTitles'].apply(lambda x: len(x.split(','))).unique())

# #Mengubah knownForTitles menjadi list of list
# name_df['knownForTitles'] = name_df['knownForTitles'].apply(lambda x: x.split(','))

# #Mencetak 5 baris teratas
# print(name_df.head())

# import numpy as np
# #menyiapkan bucket untuk dataframe
# df_uni = []

# for x in ['knownForTitles']:
#     #mengulang index dari tiap baris sampai tiap elemen dari knownForTitles
#     idx = name_df.index.repeat(name_df['knownForTitles'].str.len())
   
#    #memecah values dari list di setiap baris dan menggabungkan nya dengan rows lain menjadi dataframe
#     df1 = pd.DataFrame({
#         x: np.concatenate(name_df[x].values)
#     })
    
#     #mengganti index dataframe tersebut dengan idx yang sudah kita define di awal
#     df1.index = idx
#     #untuk setiap dataframe yang terbentuk, kita append ke dataframe bucket
#     df_uni.append(df1)
    
# #menggabungkan semua dataframe menjadi satu
# df_concat = pd.concat(df_uni, axis=1)

# #left join dengan value dari dataframe yang awal
# unnested_df = df_concat.join(name_df.drop(['knownForTitles'], 1), how='left')

# #select kolom sesuai dengan dataframe awal
# unnested_df = unnested_df[name_df.columns.tolist()]
# print(unnested_df)
# #select kolom sesuai dengan dataframe awal
# unnested_df = unnested_df[name_df.columns.tolist()]
# print(unnested_df)


# unnested_drop = unnested_df.drop(['nconst'], axis=1)

# #menyiapkan bucket untuk dataframe
# df_uni = []

# for col in ['primaryName']:
#     #agregasi kolom PrimaryName sesuai group_col yang sudah di define di atas
#     dfi = unnested_drop.groupby(['knownForTitles'])[col].apply(list)
#     #Lakukan append
#     df_uni.append(dfi)
# df_grouped = pd.concat(df_uni, axis=1).reset_index()
# df_grouped.columns = ['knownForTitles','cast_name']
# print(df_grouped)

# movie_rating_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/movie_rating_df.csv')

# #join antara movie table dan cast table 
# base_df = pd.merge(df_grouped, movie_rating_df, left_on='knownForTitles', right_on='tconst', how='inner')

# #join antara base_df dengan director_writer table
# base_df = pd.merge(base_df, director_writers, left_on='tconst', right_on='tconst', how='left')
# print(base_df.head())

# #Melakukan drop terhadap kolom knownForTitles
# base_drop = base_df.drop(['knownForTitles'], axis=1)
# print(base_drop.info())

# #Mengganti nilai NULL pada kolom genres dengan 'Unknown'
# base_drop['genres'] = base_drop['genres'].fillna('Unknown')

# #Melakukan perhitungan jumlah nilai NULL pada tiap kolom
# print(base_drop.isnull().sum())

# #Mengganti nilai NULL pada kolom dorector_name dan writer_name dengan 'Unknown'
# base_drop[['director_name','writer_name']] = base_drop[['director_name','writer_name']].fillna('unknown')

# #karena value kolom genres terdapat multiple values, jadi kita akan bungkus menjadi list of list
# base_drop['genres'] = base_drop['genres'].apply(lambda x: x.split(','))

# #Drop kolom tconst, isAdult, endYear, originalTitle
# base_drop2 = base_drop.drop(['tconst','isAdult','endYear','originalTitle'], axis=1)

# base_drop2 = base_drop2[['primaryTitle','titleType','startYear','runtimeMinutes','genres','averageRating','numVotes','cast_name','director_name','writer_name']]

# base_drop2.columns = ['title','type','start','duration','genres','rating','votes','cast_name','director_name','writer_name']
# print(base_drop2.head())

# #Klasifikasi berdasar title, cast_name, genres, director_name, dan writer_name
# feature_df = base_drop2[['title','cast_name','genres','director_name','writer_name']]

# #Tampilkan 5 baris teratas
# print(feature_df.head())

# def sanitize(x):
#     try:
#         #kalau cell berisi list
#         if isinstance(x, list):
#             return [i.replace(' ','').lower() for i in x]
#         #kalau cell berisi string
#         else:
#             return [x.replace(' ','').lower()]
#     except:
#         print(x)
        
# #Kolom : cast_name, genres, writer_name, director_name        
# feature_cols = ['cast_name','genres','writer_name','director_name']

# #Apply function sanitize 
# for col in feature_cols:
#     feature_df[col] = feature_df[col].apply(sanitize)

# #kolom yang digunakan : cast_name, genres, director_name, writer_name
# def soup_feature(x):
#     return ' '.join(x['cast_name']) + ' ' + ' '.join(x['genres']) + ' ' + ' '.join(x['director_name']) + ' ' + ' '.join(x['writer_name'])

# #membuat soup menjadi 1 kolom 
# feature_df['soup'] = feature_df.apply(soup_feature, axis=1)
    
# #import CountVectorizer 
# from sklearn.feature_extraction.text import CountVectorizer

# #definisikan CountVectorizer dan mengubah soup tadi menjadi bentuk vector
# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(feature_df['soup'])

# print(count)
# print(count_matrix.shape)

# #Import cosine_similarity
# from sklearn.metrics.pairwise import cosine_similarity

# #Gunakan cosine_similarity antara count_matrix 
# cosine_sim = cosine_similarity(count_matrix, count_matrix)

# #print hasilnya
# print(cosine_sim)

# indices = pd.Series(feature_df.index, index=feature_df['title']).drop_duplicates()

# def content_recommender(title):
#     #mendapatkan index dari judul film (title) yang disebutkan
#     idx = indices[title]

#     #menjadikan list dari array similarity cosine sim 
#     #hint: cosine_sim[idx]
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     #mengurutkan film dari similarity tertinggi ke terendah
#     sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)

#     #untuk mendapatkan list judul dari item kedua sampe ke 11
#     sim_scores = sim_scores[1:11]

#     #mendapatkan index dari judul-judul yang muncul di sim_scores
#     movie_indices = [i[0] for i in sim_scores]

#     #dengan menggunakan iloc, kita bisa panggil balik berdasarkan index dari movie_indices
#     return base_df.iloc[movie_indices]

# #aplikasikan function di atas
# print(content_recommender('The Lion King'))



In [1]:
#import library yang dibutuhkan
import pandas as pd
import numpy as np



#lakukan pembacaan dataset
movie_rating_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/movie_rating_df.csv') #untuk menyimpan movie_rating_df.csv


In [2]:
#tampilkan 5 baris teratas dari movive_rating_df
print(movie_rating_df.head())

#tampilkan info mengenai tipe data dari tiap kolom
print(movie_rating_df.info())

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

   isAdult  startYear  endYear  runtimeMinutes                    genres  \
0        0     1894.0      NaN             1.0         Documentary,Short   
1        0     1892.0      NaN             5.0           Animation,Short   
2        0     1892.0      NaN             4.0  Animation,Comedy,Romance   
3        0     1892.0      NaN            12.0           Animation,Short   
4        0     1893.0      NaN             1.0              Comedy,Short   

   averageRating  numVotes  
0            5.6      1608  
1            6.0       197  
2          

In [3]:
#Simpan actor_name.csv pada variable name_df 
name_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/actor_name.csv')

#Tampilkan 5 baris teratas dari name_df
print(name_df.head())

#Tampilkan informasi mengenai tipe data dari tiap kolom pada name_df
print(name_df.info())

       nconst          primaryName birthYear deathYear  \
0   nm1774132    Nathan McLaughlin      1973        \N   
1  nm10683464        Bridge Andrew        \N        \N   
2   nm1021485    Brandon Fransvaag        \N        \N   
3   nm6940929   Erwin van der Lely        \N        \N   
4   nm5764974  Svetlana Shypitsyna        \N        \N   

                    primaryProfession                           knownForTitles  
0  special_effects,make_up_department  tt0417686,tt1713976,tt1891860,tt0454839  
1                               actor                                tt7718088  
2                       miscellaneous                                tt0168790  
3                       miscellaneous                                tt4232168  
4                             actress                                tt3014168  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
nconst               1000 non-null object
primaryName       

In [4]:
#Menyimpan dataset pada variabel director_writers
director_writers = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/directors_writers.csv')

#Manampilkan 5 baris teratas
print(director_writers.head())

#Menampilkan informasi tipe data
print(director_writers.info())


      tconst                   director_name  \
0  tt0011414                  David Kirkland   
1  tt0011890               Roy William Neill   
2  tt0014341  Buster Keaton,John G. Blystone   
3  tt0018054                Cecil B. DeMille   
4  tt0024151                     James Cruze   

                                       writer_name  
0                          John Emerson,Anita Loos  
1     Arthur F. Goodrich,Burns Mantle,Mary Murillo  
2  Jean C. Havez,Clyde Bruckman,Joseph A. Mitchell  
3                                Jeanie Macpherson  
4                 Max Miller,Wells Root,Jack Jevne  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 3 columns):
tconst           986 non-null object
director_name    986 non-null object
writer_name      986 non-null object
dtypes: object(3)
memory usage: 23.2+ KB
None


In [5]:
import pandas as pd
director_writers = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/directors_writers.csv')

#Mengubah director_name menjadi list
director_writers['director_name'] = director_writers['director_name'].apply(lambda row: row.split(','))
director_writers['writer_name'] = director_writers['writer_name'].apply(lambda row: row.split(','))

#Tampilkan 5 data teratas
print(director_writers.head())

      tconst                      director_name  \
0  tt0011414                   [David Kirkland]   
1  tt0011890                [Roy William Neill]   
2  tt0014341  [Buster Keaton, John G. Blystone]   
3  tt0018054                 [Cecil B. DeMille]   
4  tt0024151                      [James Cruze]   

                                         writer_name  
0                         [John Emerson, Anita Loos]  
1   [Arthur F. Goodrich, Burns Mantle, Mary Murillo]  
2  [Jean C. Havez, Clyde Bruckman, Joseph A. Mitc...  
3                                [Jeanie Macpherson]  
4               [Max Miller, Wells Root, Jack Jevne]  


In [6]:
name_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/actor_name.csv')
#Kita hanya akan membutuhkan kolom nconst, primaryName, dan knownForTitles
name_df = name_df[['nconst','primaryName','knownForTitles']]

#Tampilkan 5 baris teratas dari name_df
print(name_df.head())

       nconst          primaryName                           knownForTitles
0   nm1774132    Nathan McLaughlin  tt0417686,tt1713976,tt1891860,tt0454839
1  nm10683464        Bridge Andrew                                tt7718088
2   nm1021485    Brandon Fransvaag                                tt0168790
3   nm6940929   Erwin van der Lely                                tt4232168
4   nm5764974  Svetlana Shypitsyna                                tt3014168


In [7]:
#Melakukan pengecekan variasi
print(name_df['knownForTitles'].apply(lambda x: len(x.split(','))).unique())

#Mengubah knownForTitles menjadi list of list
name_df['knownForTitles'] = name_df['knownForTitles'].apply(lambda x: x.split(','))

#Mencetak 5 baris teratas
print(name_df.head())

[4 1 2 3]
       nconst          primaryName  \
0   nm1774132    Nathan McLaughlin   
1  nm10683464        Bridge Andrew   
2   nm1021485    Brandon Fransvaag   
3   nm6940929   Erwin van der Lely   
4   nm5764974  Svetlana Shypitsyna   

                                 knownForTitles  
0  [tt0417686, tt1713976, tt1891860, tt0454839]  
1                                   [tt7718088]  
2                                   [tt0168790]  
3                                   [tt4232168]  
4                                   [tt3014168]  


In [8]:
import numpy as np
#menyiapkan bucket untuk dataframe
df_uni = []

for x in ['knownForTitles']:
    #mengulang index dari tiap baris sampai tiap elemen dari knownForTitles
    idx = name_df.index.repeat(name_df['knownForTitles'].str.len())
   
   #memecah values dari list di setiap baris dan menggabungkan nya dengan rows lain menjadi dataframe
    df1 = pd.DataFrame({
        x: np.concatenate(name_df[x].values)
    })
    
    #mengganti index dataframe tersebut dengan idx yang sudah kita define di awal
    df1.index = idx
    #untuk setiap dataframe yang terbentuk, kita append ke dataframe bucket
    df_uni.append(df1)
    
#menggabungkan semua dataframe menjadi satu
df_concat = pd.concat(df_uni, axis=1)

#left join dengan value dari dataframe yang awal
unnested_df = df_concat.join(name_df.drop(['knownForTitles'], 1), how='left')

#select kolom sesuai dengan dataframe awal
unnested_df = unnested_df[name_df.columns.tolist()]
print(unnested_df)

         nconst                primaryName knownForTitles
0     nm1774132          Nathan McLaughlin      tt0417686
0     nm1774132          Nathan McLaughlin      tt1713976
0     nm1774132          Nathan McLaughlin      tt1891860
0     nm1774132          Nathan McLaughlin      tt0454839
1    nm10683464              Bridge Andrew      tt7718088
2     nm1021485          Brandon Fransvaag      tt0168790
3     nm6940929         Erwin van der Lely      tt4232168
4     nm5764974        Svetlana Shypitsyna      tt3014168
5     nm8621807                Utku Arslan      tt5493404
5     nm8621807                Utku Arslan      tt7661932
5     nm8621807                Utku Arslan      tt0845088
5     nm8621807                Utku Arslan      tt9278408
6     nm0415875               Mihály Jakab      tt0242160
7     nm7082726             Ernesto Ballén      tt8696516
7     nm7082726             Ernesto Ballén      tt6407610
7     nm7082726             Ernesto Ballén      tt2707408
7     nm708272

In [ ]:
unnested_drop = unnested_df.drop(['nconst'], axis=1)

#menyiapkan bucket untuk dataframe
df_uni = []

for col in ['primaryName']:
    #agregasi kolom PrimaryName sesuai group_col yang sudah di define di atas
    dfi = unnested_drop.groupby(['knownForTitles'])[col].apply(list)
    #Lakukan append
    df_uni.append(dfi)
df_grouped = pd.concat(df_uni, axis=1).reset_index()
df_grouped.columns = ['knownForTitles','cast_name']
print(df_grouped)

In [ ]:
movie_rating_df = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/movie_rating_df.csv')

#join antara movie table dan cast table 
base_df = pd.merge(df_grouped, movie_rating_df, left_on='knownForTitles', right_on='tconst', how='inner')

#join antara base_df dengan director_writer table
base_df = pd.merge(base_df, director_writers, left_on='tconst', right_on='tconst', how='left')
print(base_df.head())

In [ ]:
#Melakukan drop terhadap kolom knownForTitles
base_drop = base_df.drop(['knownForTitles'], axis=1)
print(base_drop.info())

#Mengganti nilai NULL pada kolom genres dengan 'Unknown'
base_drop['genres'] = base_drop['genres'].fillna('Unknown')

#Melakukan perhitungan jumlah nilai NULL pada tiap kolom
print(base_drop.isnull().sum())

#Mengganti nilai NULL pada kolom dorector_name dan writer_name dengan 'Unknown'
base_drop[['director_name','writer_name']] = base_drop[['director_name','writer_name']].fillna('unknown')

#karena value kolom genres terdapat multiple values, jadi kita akan bungkus menjadi list of list
base_drop['genres'] = base_drop['genres'].apply(lambda x: x.split(','))

In [ ]:
#Drop kolom tconst, isAdult, endYear, originalTitle
base_drop2 = base_drop.drop(['tconst','isAdult','endYear','originalTitle'], axis=1)

base_drop2 = base_drop2[['primaryTitle','titleType','startYear','runtimeMinutes','genres','averageRating','numVotes','cast_name','director_name','writer_name']]

base_drop2.columns = ['title','type','start','duration','genres','rating','votes','cast_name','director_name','writer_name']
print(base_drop2.head())

In [ ]:
#Klasifikasi berdasar title, cast_name, genres, director_name, dan writer_name
feature_df = base_drop2[['title','cast_name','genres','director_name','writer_name']]

#Tampilkan 5 baris teratas
print(feature_df.head())

In [ ]:
def sanitize(x):
    try:
        #kalau cell berisi list
        if isinstance(x, list):
            return [i.replace(' ','').lower() for i in x]
        #kalau cell berisi string
        else:
            return [x.replace(' ','').lower()]
    except:
        print(x)
        
#Kolom : cast_name, genres, writer_name, director_name        
feature_cols = ['cast_name','genres','writer_name','director_name']

#Apply function sanitize 
for col in feature_cols:
    feature_df[col] = feature_df[col].apply(sanitize)

In [ ]:
#kolom yang digunakan : cast_name, genres, director_name, writer_name
def soup_feature(x):
    return ' '.join(x['cast_name']) + ' ' + ' '.join(x['genres']) + ' ' + ' '.join(x['director_name']) + ' ' + ' '.join(x['writer_name'])

#membuat soup menjadi 1 kolom 
feature_df['soup'] = feature_df.apply(soup_feature, axis=1)
    
    

In [ ]:
#import CountVectorizer 
from sklearn.feature_extraction.text import CountVectorizer

#definisikan CountVectorizer dan mengubah soup tadi menjadi bentuk vector
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(feature_df['soup'])

print(count)
print(count_matrix.shape)

In [ ]:
#Import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

#Gunakan cosine_similarity antara count_matrix 
cosine_sim = cosine_similarity(count_matrix, count_matrix)

#print hasilnya
print(cosine_sim)

In [ ]:
indices = pd.Series(feature_df.index, index=feature_df['title']).drop_duplicates()

def content_recommender(title):
    #mendapatkan index dari judul film (title) yang disebutkan
    idx = indices[title]

    #menjadikan list dari array similarity cosine sim 
    #hint: cosine_sim[idx]
    sim_scores = list(enumerate(cosine_sim[idx]))

    #mengurutkan film dari similarity tertinggi ke terendah
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)

    #untuk mendapatkan list judul dari item kedua sampe ke 11
    sim_scores = sim_scores[1:11]

    #mendapatkan index dari judul-judul yang muncul di sim_scores
    movie_indices = [i[0] for i in sim_scores]

    #dengan menggunakan iloc, kita bisa panggil balik berdasarkan index dari movie_indices
    return base_df.iloc[movie_indices]

#aplikasikan function di atas
print(content_recommender('The Lion King'))

